In [1]:
import pickle
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
import warnings
warnings.filterwarnings("ignore")
from tqdm.auto import tqdm
import scipy
from scipy import stats
import statsmodels.formula.api as smf
from scipy.stats.mstats import winsorize

# define data path
with open('data_path.txt') as f:
    lines = f.read().splitlines()
data_path = lines[0]

# load key data
f = open(data_path+'clean_data.pckl', 'rb')
mcdata, keydata, ids, returns, EU_4F, US_4F = pickle.load(f)
f.close()

# load betas
f = open(data_path+'betas.pckl', 'rb')
betas_onlyMkt, betas_4FBMG, betas_4F, betas_onlyBMG, betas_onlyBMG2, all_sample, bmg_sample = pickle.load(f)
f.close()

# load cross section data
f = open(data_path+'cross_section.pckl', 'rb')
cs_dict, lambdas_BMG, lambdas_BMG2, lambdas_co2 = pickle.load(f)
f.close()

# data dates
mc_date = pd.Timestamp("2021-09-30")
key_date = pd.Timestamp("2020-12-31")

# load bmg data
f = open(data_path+'bmg.pckl', 'rb')
bmg_factors_value, bmg_factors_equal, valid_dict = pickle.load(f)
f.close()


In [169]:
# Sector and Country breakdown

# list of sectors
sector_list = ids.Sector.unique()

# stats dataframe
sector_df = pd.DataFrame(columns=["Sector","Beta","Weighted_Beta","Market_Cap","CO2","CO2_Score"])
sector_df["Sector"] = sector_list
sector_df.dropna(subset = ["Sector"], inplace=True)

sector_stats = dict.fromkeys(sector_df["Sector"],0)
sector_df = sector_df.set_index("Sector")
for sector in sector_stats.keys():
    sector_stats[sector] = dict.fromkeys(list(sector_df.columns),[])


# list of countries
country_list = ids.Country.unique()

# country stats dataframe
country_stats = pd.DataFrame(columns=["Country","Average_Beta","Weighted_Average_Beta","Market_Cap","Num_Firms","Average_CO2","Average_CO2_Score","CO2counter"])
country_stats["Country"] = country_list
country_stats.dropna(subset = ["Country"], inplace=True)
country_stats = country_stats.fillna(0)
country_stats = country_stats.reset_index(drop=True)
country_stats = country_stats.set_index("Country")

# p value test
p_value_1 = 0
p_value_5 = 0
p_value_10 = 0
sum_t = 0
r2_change_list = []

# add stats from regressions
for stock in tqdm(bmg_sample):
    
    linreg = betas_4FBMG[stock]
    linreg_exbmg = betas_4F[stock]
    temp_beta = linreg[linreg.names == "BMG"]["coef"].item()
    temp_sector = ids[ids.RIC == stock]["Sector"].item()
    temp_country = ids[ids.RIC == stock]["Country"].item()
    temp_mc = mcdata[mcdata.Date == mc_date][stock].item()
    if temp_mc <= 0:
        continue
    temp_co2 = keydata[keydata.Date == key_date][keydata.RIC == stock]["TotalCO2"].item()
    temp_co2score = keydata[keydata.Date == key_date][keydata.RIC == stock]["AnalyticCO2"].item()
    
    # Sector
    sector_stats[temp_sector]["Beta"] = sector_stats[temp_sector]["Beta"] + [temp_beta]
    temp_weightedbeta = temp_beta * temp_mc
    sector_stats[temp_sector]["Weighted_Beta"] = sector_stats[temp_sector]["Weighted_Beta"] + [temp_weightedbeta]
    sector_stats[temp_sector]["Market_Cap"] = sector_stats[temp_sector]["Market_Cap"] + [temp_mc]
    
    # Country
    country_stats.loc[temp_country,"Average_Beta"] += temp_beta
    country_stats.loc[temp_country,"Weighted_Average_Beta"] += temp_beta * temp_mc
    country_stats.loc[temp_country,"Market_Cap"] += temp_mc
    country_stats.loc[temp_country,"Num_Firms"] += 1
    
    if temp_co2score > 0:
        sector_stats[temp_sector]["CO2"] = sector_stats[temp_sector]["CO2"] + [temp_co2]
        sector_stats[temp_sector]["CO2_Score"] = sector_stats[temp_sector]["CO2_Score"] + [temp_co2score]
        
        country_stats.loc[temp_country,"Average_CO2"] += temp_co2
        country_stats.loc[temp_country,"Average_CO2_Score"] += temp_co2score
        country_stats.loc[temp_country,"CO2counter"] += 1
    
    # p value test
    temp_p = linreg[linreg.names == "BMG"]["pval"].item()
    if temp_p <= 0.1:
        p_value_10 += 1
        if temp_p <= 0.05:
            p_value_5 += 1
            if temp_p <= 0.01:
                p_value_1 += 1
                
    # Rsquared Test
    temp_r2 = linreg[linreg.names == "Mkt-RF"]["adj_r2"].item()
    temp_r2exbmg = linreg_exbmg[linreg_exbmg.names == "Mkt-RF"]["adj_r2"].item()
    
    r2_change_list += [(temp_r2 - temp_r2exbmg)/temp_r2]
        

In [170]:
# calculate averages for country stats
country_stats["Weighted_Average_Beta"] = country_stats["Weighted_Average_Beta"] / country_stats["Market_Cap"]
country_stats["Average_Beta"] = country_stats["Average_Beta"] / country_stats["Num_Firms"]
country_stats["Average_CO2"] = country_stats["Average_CO2"] / country_stats["CO2counter"]
country_stats["Average_CO2_Score"] = country_stats["Average_CO2_Score"] / country_stats["CO2counter"]

In [171]:
# significant tests
total_regs = len(bmg_sample)

# winsorize r2 series at 5% and 95%
r2_change_list = winsorize(r2_change_list,limits=[0.05,0.05])


avg_r2_delta = np.sum(r2_change_list)/total_regs
median_r2_delta = np.median(r2_change_list)
# p value tests
print("Total amount of regressions: " +str(total_regs))
print("Amount of significant BMG coefficient at 10% level: "+str(p_value_10)+", equal to "+str(100*round(p_value_10/total_regs,4))+"%")
print("Amount of significant BMG coefficient at 5% level: "+str(p_value_5)+", equal to "+str(100*round(p_value_5/total_regs,4))+"%")
print("Amount of significant BMG coefficient at 1% level: "+str(p_value_1)+", equal to "+str(100*round(p_value_1/total_regs,4))+"%")

# winsorized mean
print("Average change in adjusted R squared: "+  str(round(100*round(avg_r2_delta,4),2))+"%")
print("Median change in adjusted R squared: "+  str(round(100*round(median_r2_delta,4),2))+"%")



Total amount of regressions: 1709
Amount of significant BMG coefficient at 10% level: 402, equal to 23.52%
Amount of significant BMG coefficient at 5% level: 277, equal to 16.21%
Amount of significant BMG coefficient at 1% level: 138, equal to 8.07%
Average change in adjusted R squared: 5.0%
Median change in adjusted R squared: 0.05%


In [148]:
# Cross Section Results
lambdasBMG_df = pd.DataFrame(columns=["Date","Constant","Mkt","BMG","log_bm","log_size","mom"])
lambdasBMG2_df = pd.DataFrame(columns=["Date","Constant","Mkt2","BMG2","log_bm","log_size","mom"])
lambdasco2_df = pd.DataFrame(columns=["Date","Constant","Mkt","co2","log_bm","log_size","mom"])
for t in lambdas_BMG.keys():
    
    # BMG
    lreg = lambdas_BMG[t]
    
    tmp_const = lreg[lreg.names == "Intercept"]["coef"].item()
    tmp_mkt = lreg[lreg.names == "Mkt"]["coef"].item()
    tmp_value = lreg[lreg.names == "log_bm"]["coef"].item()
    tmp_smb = lreg[lreg.names == "log_size"]["coef"].item()
    tmp_wml = lreg[lreg.names == "logmom"]["coef"].item()
    tmp_bmg = lreg[lreg.names == "BMG"]["coef"].item()
    
    
    temp_df = pd.DataFrame({
    "Date":[t],
    "Constant":[tmp_const],
    "Mkt":[tmp_mkt],
    "log_bm":[tmp_value],
    "log_size":[tmp_smb],
    "mom":[tmp_wml],
    "BMG":[tmp_bmg]})

    lambdasBMG_df = lambdasBMG_df.append(temp_df)
    
    # BMG2
    lreg = lambdas_BMG2[t]
    
    tmp_const = lreg[lreg.names == "Intercept"]["coef"].item()
    tmp_mkt = lreg[lreg.names == "Mkt2"]["coef"].item()
    tmp_value = lreg[lreg.names == "log_bm"]["coef"].item()
    tmp_smb = lreg[lreg.names == "log_size"]["coef"].item()
    tmp_wml = lreg[lreg.names == "logmom"]["coef"].item()
    tmp_bmg = lreg[lreg.names == "BMG2"]["coef"].item()
    
    
    temp_df = pd.DataFrame({
    "Date":[t],
    "Constant":[tmp_const],
    "Mkt2":[tmp_mkt],
    "log_bm":[tmp_value],
    "log_size":[tmp_smb],
    "mom":[tmp_wml],
    "BMG2":[tmp_bmg]})

    lambdasBMG2_df = lambdasBMG2_df.append(temp_df)
    
    
    # CO2
    lreg = lambdas_co2[t]
    
    tmp_const = lreg[lreg.names == "Intercept"]["coef"].item()
    tmp_mkt = lreg[lreg.names == "Mkt"]["coef"].item()
    tmp_value = lreg[lreg.names == "log_bm"]["coef"].item()
    tmp_smb = lreg[lreg.names == "log_size"]["coef"].item()
    #tmp_wml = lreg[lreg.names == "mom"]["coef"].item()
    tmp_wml = lreg[lreg.names == "logmom"]["coef"].item()
    tmp_co2 = lreg[lreg.names == "log_co2score"]["coef"].item()
    
    
    temp_df = pd.DataFrame({
    "Date":[t],
    "Constant":[tmp_const],
    "Mkt":[tmp_mkt],
    "log_bm":[tmp_value],
    "log_size":[tmp_smb],
    "mom":[tmp_wml],
    "co2":[tmp_co2]})

    lambdasco2_df = lambdasco2_df.append(temp_df)


lambdasBMG_df = lambdasBMG_df.reset_index(drop=True)  
lambdasBMG2_df = lambdasBMG2_df.reset_index(drop=True)  
lambdasco2_df = lambdasco2_df.reset_index(drop=True)  

In [151]:
# best amount of lags: many practitioners simply set it to the integer part of T^(0.25) see e.g. Greene (Econometric Analysis, 7th edition, section 20.5.2, p. 960).

#lambdasBMG results
lambdasBMG_stats = pd.DataFrame(columns=["var","coef","std err","t","p"])
months_amount = len(lambdasBMG_df)
num_lags = int(np.power(months_amount,0.25))
for reg in list(lambdasBMG_df.columns)[1:]:
    reg_string = str(reg)
    
    # Newey West Standard Errors
    lm = smf.ols(reg_string+' ~ 1',data=lambdasBMG_df).fit(cov_type='HAC',cov_kwds={'maxlags':num_lags})
    stats = pd.read_html(lm.summary().as_html())[1]
    
    
        
    temp_df = pd.DataFrame({
    "var":[reg],
    "coef":[stats.loc[1,1]],
    "std err":[stats.loc[1,2]],
    "t":[stats.loc[1,3]],
    "p":[stats.loc[1,4]]})

    lambdasBMG_stats = lambdasBMG_stats.append(temp_df)

lambdasBMG_stats = lambdasBMG_stats.reset_index(drop=True)
lambdasBMG_stats

,var,coef,std err,t,p
0,Constant,1.5561,0.494,3.150,0.002
1,Mkt,-0.0495,0.319,-0.155,0.877
2,BMG,-0.2806,0.193,-1.452,0.147
3,log_bm,-0.0143,0.080,-0.178,0.859
4,log_size,-0.0653,0.046,-1.429,0.153
5,mom,0.7135,0.367,1.942,0.052


In [20]:
# best amount of lags: many practitioners simply set it to the integer part of T^(0.25) see e.g. Greene (Econometric Analysis, 7th edition, section 20.5.2, p. 960).

#lambdasBMG2 results
lambdasBMG2_stats = pd.DataFrame(columns=["var","coef","std err","t","p"])
months_amount = len(lambdasBMG2_df)
num_lags = int(np.power(months_amount,0.25))
for reg in list(lambdasBMG2_df.columns)[1:]:
    reg_string = str(reg)
    
    # Newey West Standard Errors
    lm = smf.ols(reg_string+' ~ 1',data=lambdasBMG2_df).fit(cov_type='HAC',cov_kwds={'maxlags':num_lags})
    stats = pd.read_html(lm.summary().as_html())[1]
    
    
        
    temp_df = pd.DataFrame({
    "var":[reg],
    "coef":[stats.loc[1,1]],
    "std err":[stats.loc[1,2]],
    "t":[stats.loc[1,3]],
    "p":[stats.loc[1,4]]})

    lambdasBMG2_stats = lambdasBMG2_stats.append(temp_df)

lambdasBMG2_stats = lambdasBMG2_stats.reset_index(drop=True)
lambdasBMG2_stats

,var,coef,std err,t,p
0,Constant,1.9569,0.844,2.318,0.020
1,Mkt2,0.0546,0.330,0.166,0.869
2,BMG2,-0.3094,0.198,-1.563,0.118
3,log_bm,-0.0263,0.078,-0.338,0.736
4,log_size,-0.0993,0.047,-2.114,0.035
5,mom,-0.2671,0.643,-0.415,0.678


In [149]:
# best amount of lags: many practitioners simply set it to the integer part of T^(0.25) see e.g. Greene (Econometric Analysis, 7th edition, section 20.5.2, p. 960).

#lambdasco2 results
lambdasco2_stats = pd.DataFrame(columns=["var","coef","std err","t","p"])
months_amount = len(lambdasco2_df)
num_lags = int(np.power(months_amount,0.25))
for reg in list(lambdasco2_df.columns)[1:]:
    reg_string = str(reg)
    
    # Newey West Standard Errors
    lm = smf.ols(reg_string+' ~ 1',data=lambdasco2_df).fit(cov_type='HAC',cov_kwds={'maxlags':num_lags})
    stats = pd.read_html(lm.summary().as_html())[1]
    
    
        
    temp_df = pd.DataFrame({
    "var":[reg],
    "coef":[stats.loc[1,1]],
    "std err":[stats.loc[1,2]],
    "t":[stats.loc[1,3]],
    "p":[stats.loc[1,4]]})

    lambdasco2_stats = lambdasco2_stats.append(temp_df)

lambdasco2_stats = lambdasco2_stats.reset_index(drop=True)
lambdasco2_stats

,var,coef,std err,t,p
0,Constant,1.9202,0.557,3.445,0.001
1,Mkt,-0.0560,0.316,-0.177,0.859
2,co2,-0.0633,0.031,-2.054,0.040
3,log_bm,-0.0059,0.082,-0.072,0.943
4,log_size,-0.0745,0.046,-1.624,0.104
5,mom,0.7739,0.401,1.929,0.054


In [172]:
# Save Data for use in other files
f = open(data_path+'results.pckl', 'wb')
pickle.dump([country_stats, sector_stats], f)
f.close()

In [167]:
########## DEVVVVVVVVVVVV #############
# load cross section data
f = open(data_path+'cross_section.pckl', 'rb')
cs_dict, lambdas_BMG, lambdas_BMG2, lambdas_co2 = pickle.load(f)
f.close()

# Different control variable models
lambdasco2_df = pd.DataFrame()
for t in lambdas_co2.keys():

    # CO2
    lreg = lambdas_co2[t]
    
    tmp_const = lreg[lreg.names == "Intercept"]["coef"].item()
    tmp_mkt = lreg[lreg.names == "Mkt"]["coef"].item()
    tmp_value = lreg[lreg.names == "log_bm"]["coef"].item()
    tmp_smb = lreg[lreg.names == "log_size"]["coef"].item()
    tmp_wml = lreg[lreg.names == "logmom"]["coef"].item()
    tmp_co2 = lreg[lreg.names == "log_co2score"]["coef"].item()
    
    
    temp_df = pd.DataFrame({
    "Date":[t],
    "Constant":[tmp_const],
    "Mkt":[tmp_mkt],
    "log_bm":[tmp_value],
    "log_size":[tmp_smb],
    "logmom":[tmp_wml],
    "co2":[tmp_co2]})

    lambdasco2_df = lambdasco2_df.append(temp_df)

lambdasco2_df = lambdasco2_df.reset_index(drop=True)


# best amount of lags: many practitioners simply set it to the integer part of T^(0.25) see e.g. Greene (Econometric Analysis, 7th edition, section 20.5.2, p. 960).

#lambdasco2 results
lambdasco2_stats = pd.DataFrame(columns=["var","coef","std err","t","p"])
months_amount = len(lambdasco2_df)
num_lags = int(np.power(months_amount,0.25))
for reg in list(lambdasco2_df.columns)[1:]:
    reg_string = str(reg)
    
    # Newey West Standard Errors
    lm = smf.ols(reg_string+' ~ 1',data=lambdasco2_df).fit(cov_type='HAC',cov_kwds={'maxlags':num_lags})
    stats = pd.read_html(lm.summary().as_html())[1]
    
    
        
    temp_df = pd.DataFrame({
    "var":[reg],
    "coef":[stats.loc[1,1]],
    "std err":[stats.loc[1,2]],
    "t":[stats.loc[1,3]],
    "p":[stats.loc[1,4]]})

    lambdasco2_stats = lambdasco2_stats.append(temp_df)

lambdasco2_stats = lambdasco2_stats.reset_index(drop=True)
lambdasco2_stats

########## DEVVVVVVVVVVVV #############

,var,coef,std err,t,p
0,Constant,2.1134,0.513,4.117,0.000
1,Mkt,-0.0569,0.315,-0.180,0.857
2,log_bm,-0.0461,0.085,-0.540,0.590
3,log_size,-0.1086,0.043,-2.510,0.012
4,logmom,0.2510,0.544,0.461,0.644
5,co2,-0.0661,0.028,-2.324,0.020
